# NameForge: AI Domain Name Generator Homework

**Author:** Ferdinand Koenig
**Date:** Sep 2025

---

## Introduction

This notebook documents the AI Engineer homework assignment for building a domain name generator.
Objectives:

- Generate synthetic dataset of business descriptions → domain names
- Build a baseline domain generator (mock / open-source LLM)
- Implement evaluation framework (LLM-as-a-judge / safety checks)
- Analyze edge cases and iteratively improve
- Ensure safety guardrails for inappropriate content


In [1]:
import pandas as pd

## 1️⃣ Step: Synthetic Dataset Creation

### 1.1 Methodology

**Objective:**
Generate a synthetic dataset of business descriptions mapped to domain names, with diversity in business types, complexity levels, and edge cases, while ensuring safety and reproducibility.

**Steps Taken:**

1. **Vocabulary Selection**
   - **Business types:** cafe, restaurant, tech startup, online store, boutique, law firm, travel agency, bookstore, etc.
   - **Adjectives / descriptors:** organic, eco-friendly, bright, cozy, modern, smart, fresh, premium, global, innovative
   - **Nouns / themes:** hub, shop, store, lab, studio, solutions, works, spot, corner
   - **TLDs:** .com, .net, .org, .io, .co
   - Vocabulary is stored externally in `src/vocab.py` for maintainability and easy updates.

2. **Complexity Levels**
   - **Simple:** Short, straightforward descriptions → short domains (e.g., “Organic cafe”)
   - **Medium:** Include location or moderate complexity (e.g., “Organic cafe in downtown area”)
   - **Complex:** Long or multi-purpose descriptions (e.g., “Premium organic cafe in downtown area offering community events for busy professionals”)

3. **Edge Cases**
   - ~5% of entries include unusual or extreme cases:
     - Extremely long business descriptions
     - Very short or ambiguous descriptions
     - Uncommon characters (e.g., symbols @$%^)
   - These cases test model robustness and evaluation coverage.

4. **Safety Guardrails**
   - Forbidden words: `adult`, `nude`, `porn`, `illegal`
   - Any generated domain containing forbidden words is replaced with `blocked.com`

5. **Dataset Generation Process**
   - Randomly combine adjectives, nouns, and business types according to complexity
   - Assign complexity distribution: 40% simple, 40% medium, 20% complex
   - Randomly insert edge cases
   - Save datasets as CSV with fields: `business_description`, `domain_name`, `complexity`

6. **Train/Test Split**
   - Train dataset: 500 entries
   - Test dataset: 100 entries
   - Stored in `data/raw/` as `train_dataset.csv` and `test_dataset.csv`

### 1.2 Practical Considerations

- **Reflecting client needs:**
  - The synthetic dataset is designed to mimic the examples provided in the homework task, ensuring generated domain names are relevant to realistic business descriptions.

- **Resource efficiency:**
  - No fine-tuned LLM is used at this stage due to GPU requirements.
  - No external API calls (OpenAI, Claude, etc.) are used because free accounts may not have access or sufficient credits.

- **Reproducibility:**
  - Dataset generation relies on deterministic Python code with controlled randomness (`random` module).
  - Vocabulary is externalized for maintainability (`src/vocab.py`).
  - The process can be fully run on a standard laptop without specialized hardware.

- **Edge cases and safety:**
  - ~5% of entries are extreme or unusual to test model robustness.
  - Safety guardrails ensure forbidden words are blocked.



In [1]:
from src.data_utils import generate_train_test

generate_train_test(train_size=3_000, test_size=100)

generating 3000 entries
Dataset generated at data/raw/train_dataset.csv with 3000 entries
generating 100 entries
Dataset generated at data/raw/test_dataset.csv with 100 entries
